In [2]:
import pandas as pd
import gym
import numpy as np
from gym.spaces import Box, MultiDiscrete
from stable_baselines3 import A2C, DDPG, PPO
from gym.spaces import Discrete, Box, MultiDiscrete, Dict
from stable_baselines3.common.env_checker import check_env
import random
from getdata import get_data
import matplotlib.pyplot as plt

In [3]:
bitcoin_data = get_data('BTC-USD', "2021-10-01", "2021-11-20", "5m", excel = True, output = "Bitcoin_1W_5m.xlsx")

[*********************100%***********************]  1 of 1 completed


In [4]:
bitcoin_data.drop(columns = "Datetime", inplace = True)
bitcoin_data.head()

,Open,High,Low,Close,Adj Close,Volume
0,43426.773438,43480.832031,43392.843750,43478.886719,43478.886719,0
1,43503.757812,43503.796875,43481.054688,43503.796875,43503.796875,27355136
2,43498.382812,43498.382812,43449.058594,43481.453125,43481.453125,0
3,43454.011719,43463.593750,43439.933594,43441.437500,43441.437500,24281088
4,43449.937500,43449.937500,43419.835938,43419.835938,43419.835938,0


In [5]:
l = np.array([1] + [1] + bitcoin_data["Open"].iloc[20-19:20].tolist() + bitcoin_data["High"].iloc[20-19:20].tolist() + bitcoin_data["Low"].iloc[20-19:20].tolist() + bitcoin_data["Close"].iloc[20-19:20].tolist() + bitcoin_data["Adj Close"].iloc[20-19:20].tolist() + bitcoin_data["Volume"].iloc[20-19:20].tolist())
print(l.shape)
print(type(l[0]))
print(type(l))

(116,)
<class 'numpy.float64'>
<class 'numpy.ndarray'>


In [6]:
class BitTrader(gym.Env):
    def __init__(self, data, initial_balance = 40):
        self.iter = 30
        self.initial_balance = initial_balance
        self.initial_data = data
        self.action_space = Box(low = -0.25, high = 0.25, shape = (1,)) 
        spaces = {
            'Balance' : Box(low = 0, high = 10000, shape = (1,)),
            'Bitcoins' : Box(low = 0, high = 5, shape = (1,)),
            'Open' : Box(low = 0, high = 100000, shape = (30,)),
            'High' : Box(low = 0, high = 100000, shape = (30,)),
            'Low' : Box(low = 0, high = 100000, shape = (30,)),
            'Close': Box(low = 0, high = 100000, shape = (30,)),
            'Adj Close': Box(low = 0, high = 100000, shape = (30,)),
            'Volume': Box(low = 0, high = 99999999, shape = (30,))
        }
        #self.observation_space = Box(low = 0, high = 99999999, shape = (122,))
        self.observation_space = Dict(spaces)
        
    def reset(self):
        self.balance = self.initial_balance
        self.net_worth = self.balance
        self.iter = 30
        self.data = self.initial_data
        self.bitcoins = 0
        #self.state = np.array([self.balance] + [self.bitcoins] + self.data["Open"].iloc[0:19].tolist() + self.data["High"].iloc[0:19].tolist() + self.data["Low"].iloc[0:19].tolist()) + self.data["Close"].iloc[0:19].tolist() + self.data["Adj Close"].iloc[0:19].tolist() + self.data["Volume"].iloc[0:19].tolist()
        self.state = {
            'Balance' : np.array([self.balance]),
            'Bitcoins' : np.array([self.bitcoins]),
            'Open' : np.array(self.data["Open"].iloc[0:30].tolist()),
            'High' : np.array(self.data["High"].iloc[0:30].tolist()),
            'Low' : np.array(self.data["Low"].iloc[0:30].tolist()),
            'Close' : np.array(self.data["Close"].iloc[0:30].tolist()),
            'Adj Close': np.array(self.data["Adj Close"].iloc[0:30].tolist()),
            'Volume': np.array(self.data["Volume"].iloc[0:30].tolist())
        }

        return self.state

    def step(self, action):
        self._take_action(action)

        self.net_worth = self.balance + self.bitcoins*self.data["Close"].iloc[self.iter]
        reward = self.net_worth

        self.state = {
            'Balance' : np.array([self.balance]),
            'Bitcoins' : np.array([self.bitcoins]),
            'Open' : np.array(self.data["Open"].iloc[self.iter-30:self.iter].tolist()),
            'High' : np.array(self.data["High"].iloc[self.iter-30:self.iter].tolist()),
            'Low' : np.array(self.data["Low"].iloc[self.iter-30:self.iter].tolist()),
            'Close' : np.array(self.data["Close"].iloc[self.iter-30:self.iter].tolist()),
            'Adj Close': np.array(self.data["Adj Close"].iloc[self.iter-30:self.iter].tolist()),
            'Volume': np.array(self.data["Volume"].iloc[self.iter-30:self.iter].tolist())
        }

        penalty = 0

        if action[0] == 0:
            penalty += 0.05*reward
        reward -= 0.05

        if self.net_worth <= 0 or self.balance < 0:
            done = True
        else:
            done = self.iter >= len(self.data.index.unique())-1
        self.iter += 1

        return self.state, reward, done, {}

    def _take_action(self, action):

        if action[0] > 0:
            amount = self.balance*action[0]
            #print(action[0])
            self.balance -= amount
            self.bitcoins += amount/self.data["Close"].iloc[self.iter]

        if action[0] < 0:
            amount = -self.bitcoins*action[0]
            #print(action[0])
            self.bitcoins -= amount
            self.balance += amount*self.data["Close"].iloc[self.iter]


    def render(self):
        print("----------------------------------------------------------------------------------------------")
        print('Net Worth: ' + str(self.net_worth))
        print('Balance: ' + str(self.balance))
        print('Bitcoins: ' + str(self.bitcoins))

In [7]:
train_df = bitcoin_data.iloc[:int(0.8*len(bitcoin_data.index.unique()))]

In [8]:
env = BitTrader(train_df)
check_env(env, warn = True)

In [9]:
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

In [10]:
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

In [10]:
model = DDPG("MultiInputPolicy", env, action_noise=action_noise, verbose=1, gamma = 0.7, learning_rate = 0.00075)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [14]:
model.learn(total_timesteps=500000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.15e+04 |
|    ep_rew_mean     | 5.88e+05 |
| time/              |          |
|    episodes        | 4        |
|    fps             | 135      |
|    time_elapsed    | 338      |
|    total timesteps | 45840    |
| train/             |          |
|    actor_loss      | -10.8    |
|    critic_loss     | 1.09e+05 |
|    learning_rate   | 0.00075  |
|    n_updates       | 34380    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.15e+04 |
|    ep_rew_mean     | 5.89e+05 |
| time/              |          |
|    episodes        | 8        |
|    fps             | 109      |
|    time_elapsed    | 839      |
|    total timesteps | 91680    |
| train/             |          |
|    actor_loss      | -40.3    |
|    critic_loss     | 1.56e+03 |
|    learning_rate   | 0.00075  |
|    n_updates       | 80220    |
--------------

In [11]:
test_df = bitcoin_data.iloc[int(0.8*len(bitcoin_data.index.unique())):]
env = BitTrader(test_df)

In [16]:
obs = env.reset()
dones = False
while not dones:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

----------------------------------------------------------------------------------------------
Net Worth: 40.0
Balance: 30.0
Bitcoins: 0.00014854952414829187
----------------------------------------------------------------------------------------------
Net Worth: 39.98452415699596
Balance: 22.5
Bitcoins: 0.0002601343541906476
----------------------------------------------------------------------------------------------
Net Worth: 40.01046036502862
Balance: 16.875
Bitcoins: 0.0003436990185009455
----------------------------------------------------------------------------------------------
Net Worth: 39.9931518972688
Balance: 12.65625
Bitcoins: 0.0004064194401308495
----------------------------------------------------------------------------------------------
Net Worth: 39.91134410918309
Balance: 19.470023527295773
Bitcoins: 0.0003048145800981371
----------------------------------------------------------------------------------------------
Net Worth: 39.88925219621769
Balance: 14.6025176

----------------------------------------------------------------------------------------------
Net Worth: 40.35997924422489
Balance: 0.17745916154316638
Bitcoins: 0.000588814792460366
----------------------------------------------------------------------------------------------
Net Worth: 40.41814310544262
Balance: 0.13309437115737477
Bitcoins: 0.0005894639525213624
----------------------------------------------------------------------------------------------
Net Worth: 40.389531077809686
Balance: 0.09982077836803108
Bitcoins: 0.0005899511686071555
----------------------------------------------------------------------------------------------
Net Worth: 40.467132701449046
Balance: 0.07486558377602331
Bitcoins: 0.000590315878207845
----------------------------------------------------------------------------------------------
Net Worth: 40.46727105673301
Balance: 0.056149187832017486
Bitcoins: 0.0005905894094714379
--------------------------------------------------------------------------

----------------------------------------------------------------------------------------------
Net Worth: 38.49662096682754
Balance: 3.629149366578697
Bitcoins: 0.0005386681155950695
----------------------------------------------------------------------------------------------
Net Worth: 38.51508928733762
Balance: 2.7218620249340226
Bitcoins: 0.0005526773916454527
----------------------------------------------------------------------------------------------
Net Worth: 38.432792169487904
Balance: 2.041396518700517
Bitcoins: 0.0005632085623421418
----------------------------------------------------------------------------------------------
Net Worth: 38.41276526502306
Balance: 1.5310473890253877
Bitcoins: 0.0005711112893915866
----------------------------------------------------------------------------------------------
Net Worth: 38.48420771788039
Balance: 1.1482855417690407
Bitcoins: 0.0005770268757763804
---------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------
Net Worth: 38.62152257728655
Balance: 0.005469952343837369
Bitcoins: 0.0005963063671591014
----------------------------------------------------------------------------------------------
Net Worth: 38.6287900611363
Balance: 0.004102464257878027
Bitcoins: 0.000596327479840517
----------------------------------------------------------------------------------------------
Net Worth: 38.60882240817852
Balance: 0.00307684819340852
Bitcoins: 0.0005963433225417178
----------------------------------------------------------------------------------------------
Net Worth: 38.699813683654305
Balance: 9.677261057058633
Bitcoins: 0.00044725749190628833
----------------------------------------------------------------------------------------------
Net Worth: 38.75978287666088
Balance: 7.257945792793975
Bitcoins: 0.00048446392413859494
----------------------------------------------------------------------------

Bitcoins: 0.0005980469581634723
----------------------------------------------------------------------------------------------
Net Worth: 38.25057375348113
Balance: 0.01937430671684638
Bitcoins: 0.0005981479987006652
----------------------------------------------------------------------------------------------
Net Worth: 38.279922726182996
Balance: 9.584511411583383
Bitcoins: 0.0004486109990254989
----------------------------------------------------------------------------------------------
Net Worth: 38.199050079288355
Balance: 16.738146078509626
Bitcoins: 0.00033645824926912415
----------------------------------------------------------------------------------------------
Net Worth: 38.20157483044791
Balance: 22.104003266494196
Bitcoins: 0.0002523436869518431
----------------------------------------------------------------------------------------------
Net Worth: 38.1506654773229
Balance: 16.578002449870645
Bitcoins: 0.00033924346722045685
---------------------------------------------

...

----------------------------------------------------------------------------------------------
Net Worth: 36.671990669708286
Balance: 1.6812735983580636
Bitcoins: 0.0005829718450295531
----------------------------------------------------------------------------------------------
Net Worth: 36.65710894702365
Balance: 1.2609551987685477
Bitcoins: 0.0005899776474776116
----------------------------------------------------------------------------------------------
Net Worth: 36.527219375970965
Balance: 0.9457163990764108
Bitcoins: 0.0005952513516755248
----------------------------------------------------------------------------------------------
Net Worth: 36.456798350827036
Balance: 0.709287299307308
Bitcoins: 0.0005992144734245102
----------------------------------------------------------------------------------------------
Net Worth: 36.47758828423702
Balance: 0.531965474480481
Bitcoins: 0.0006021850870953986
-------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------
Net Worth: 36.59590460498452
Balance: 6.872407046073427
Bitcoins: 0.0004977598415718404
----------------------------------------------------------------------------------------------
Net Worth: 36.51842323027047
Balance: 5.15430528455507
Bitcoins: 0.0005266069573294559
----------------------------------------------------------------------------------------------
Net Worth: 36.55124565453277
Balance: 3.8657289634163026
Bitcoins: 0.0005482196765232833
----------------------------------------------------------------------------------------------
Net Worth: 36.559481798579284
Balance: 2.899296722562227
Bitcoins: 0.0005644251324442952
----------------------------------------------------------------------------------------------
Net Worth: 36.675616679151055
Balance: 2.1744725419216704
Bitcoins: 0.0005765374343375354
----------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------
Net Worth: 36.56824768022445
Balance: 1.232714343478021
Bitcoins: 0.0005925225843166848
----------------------------------------------------------------------------------------------
Net Worth: 36.57781136506194
Balance: 0.9245357576085158
Bitcoins: 0.0005976888672673004
----------------------------------------------------------------------------------------------
Net Worth: 36.57596459985096
Balance: 0.6934018182063868
Bitcoins: 0.0006015637801926584
----------------------------------------------------------------------------------------------
Net Worth: 36.631928830277005
Balance: 9.678033571224042
Bitcoins: 0.00045117283514449377
----------------------------------------------------------------------------------------------
Net Worth: 36.66842448289487
Balance: 7.258525178418031
Bitcoins: 0.0004916174683775072
---------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------
Net Worth: 35.835435630468844
Balance: 0.10521620442875382
Bitcoins: 0.0006140138975444349
----------------------------------------------------------------------------------------------
Net Worth: 35.84674451924588
Balance: 0.07891215332156537
Bitcoins: 0.0006144657823323285
----------------------------------------------------------------------------------------------
Net Worth: 35.78289768405041
Balance: 0.05918411499117403
Bitcoins: 0.000614805301977723
----------------------------------------------------------------------------------------------
Net Worth: 35.71158987535384
Balance: 0.04438808624338052
Bitcoins: 0.0006150604510126437
----------------------------------------------------------------------------------------------
Net Worth: 35.679986339523296
Balance: 0.033291064682535385
Bitcoins: 0.0006152519824985922
-------------------------------------------------------------------------

----------------------------------------------------------------------------------------------
Net Worth: 35.2981834557861
Balance: 8.682440936072204
Bitcoins: 0.0004656011528351967
----------------------------------------------------------------------------------------------
Net Worth: 35.287603760840625
Balance: 6.511830702054153
Bitcoins: 0.0005035877133374243
----------------------------------------------------------------------------------------------
Net Worth: 35.25533873867594
Balance: 4.8838730265406145
Bitcoins: 0.0005321096140802789
----------------------------------------------------------------------------------------------
Net Worth: 35.120228624869505
Balance: 3.662904769905461
Bitcoins: 0.0005535966264859113
----------------------------------------------------------------------------------------------
Net Worth: 35.177506413329404
Balance: 2.7471785774290955
Bitcoins: 0.0005696825963063639
---------------------------------------------------------------------------------

In [17]:
#model.save('ddpg_bitcoin_hist')

In [11]:
model = DDPG.load("ddpg_bitcoin_hist")
env = BitTrader(bitcoin_data)
obs = env.reset()
dones = False
reward = []
while not dones:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    reward.append(rewards)
    env.render()

----------------------------------------------------------------------------------------------
Net Worth: 40.0
Balance: 30.0
Bitcoins: 0.00022782708493820903
----------------------------------------------------------------------------------------------
Net Worth: 39.9851609811943
Balance: 22.5
Bitcoins: 0.0003989513302312673
----------------------------------------------------------------------------------------------
Net Worth: 39.99023670182931
Balance: 16.875
Bitcoins: 0.0005272572686132314
----------------------------------------------------------------------------------------------
Net Worth: 39.987756944987865
Balance: 12.65625
Bitcoins: 0.0006234970468126226
----------------------------------------------------------------------------------------------
Net Worth: 39.99065279650607
Balance: 19.489850699126517
Bitcoins: 0.0004676227851094669
----------------------------------------------------------------------------------------------
Net Worth: 40.05716483108187
Balance: 14.617388

----------------------------------------------------------------------------------------------
Net Worth: 42.285056211145886
Balance: 2.6631211218871678
Bitcoins: 0.0008387378155184976
----------------------------------------------------------------------------------------------
Net Worth: 42.27583992413396
Balance: 1.997340841415376
Bitcoins: 0.0008528346792264875
----------------------------------------------------------------------------------------------
Net Worth: 42.171787430497396
Balance: 1.498005631061532
Bitcoins: 0.0008634347103433901
----------------------------------------------------------------------------------------------
Net Worth: 42.16411432906759
Balance: 11.664532805563045
Bitcoins: 0.0006475760327575425
----------------------------------------------------------------------------------------------
Net Worth: 42.1633554509042
Balance: 8.748399604172285
Bitcoins: 0.0007094937661569217
----------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------
Net Worth: 42.49155490836808
Balance: 3.4685247456802606
Bitcoins: 0.0008215674075665817
----------------------------------------------------------------------------------------------
Net Worth: 42.53340670743869
Balance: 2.6013935592601953
Bitcoins: 0.0008398039075909427
----------------------------------------------------------------------------------------------
Net Worth: 42.49530388561576
Balance: 1.9510451694451465
Bitcoins: 0.0008534943459208584
----------------------------------------------------------------------------------------------
Net Worth: 42.48815587046868
Balance: 1.4632838770838599
Bitcoins: 0.0008637639852214931
----------------------------------------------------------------------------------------------
Net Worth: 42.570628460666995
Balance: 1.0974629078128948
Bitcoins: 0.0008714507619154387
-------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------
Net Worth: 42.68098388273687
Balance: 0.003382626888338382
Bitcoins: 0.0008923672101091888
----------------------------------------------------------------------------------------------
Net Worth: 42.68514245336836
Balance: 0.0025369701662537867
Bitcoins: 0.0008923848906437512
----------------------------------------------------------------------------------------------
Net Worth: 42.700055041501855
Balance: 10.676916488000153
Bitcoins: 0.0006692886679828135
----------------------------------------------------------------------------------------------
Net Worth: 42.695388321687986
Balance: 8.007687366000114
Bitcoins: 0.0007250841104662587
----------------------------------------------------------------------------------------------
Net Worth: 42.725482144632146
Balance: 6.005765524500086
Bitcoins: 0.000766894419183229
---------------------------------------------------------------------------

----------------------------------------------------------------------------------------------
Net Worth: 42.63387293442926
Balance: 0.3562643484691337
Bitcoins: 0.0008833839882007633
----------------------------------------------------------------------------------------------
Net Worth: 42.580355738050336
Balance: 0.26719826135185026
Bitcoins: 0.0008852473688600453
----------------------------------------------------------------------------------------------
Net Worth: 42.58284549627525
Balance: 0.2003986960138877
Bitcoins: 0.0008866448221266379
----------------------------------------------------------------------------------------------
Net Worth: 42.646008549482765
Balance: 0.15029902201041578
Bitcoins: 0.0008876913524203188
----------------------------------------------------------------------------------------------
Net Worth: 42.67512551935102
Balance: 0.11272426650781184
Bitcoins: 0.0008884757127170281
---------------------------------------------------------------------------

...

----------------------------------------------------------------------------------------------
Net Worth: 52.11926213007399
Balance: 0.18585476035101234
Bitcoins: 0.0008826400696826145
----------------------------------------------------------------------------------------------
Net Worth: 52.32139704946963
Balance: 0.13939107026325925
Bitcoins: 0.0008834266868895352
----------------------------------------------------------------------------------------------
Net Worth: 52.525030351683164
Balance: 0.10454330269744444
Bitcoins: 0.0008840143564926641
----------------------------------------------------------------------------------------------
Net Worth: 52.93024043234049
Balance: 0.07840747702308333
Bitcoins: 0.0008844517278170448
----------------------------------------------------------------------------------------------
Net Worth: 52.515753876723544
Balance: 0.0588056077673125
Bitcoins: 0.0008847823491837242
--------------------------------------------------------------------------

----------------------------------------------------------------------------------------------
Net Worth: 53.645934686007024
Balance: 3.1748041984225144
Bitcoins: 0.0008376779498566831
----------------------------------------------------------------------------------------------
Net Worth: 53.52737707866637
Balance: 2.381103148816886
Bitcoins: 0.0008508821583012848
----------------------------------------------------------------------------------------------
Net Worth: 53.66840082512951
Balance: 1.7858273616126645
Bitcoins: 0.0008607580841064917
----------------------------------------------------------------------------------------------
Net Worth: 53.57058373847784
Balance: 1.3393705212094984
Bitcoins: 0.000868179019560643
----------------------------------------------------------------------------------------------
Net Worth: 53.55869036417473
Balance: 1.0045278909071238
Bitcoins: 0.0008737459887850073
---------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------
Net Worth: 52.90988800196317
Balance: 5.61016098479981
Bitcoins: 0.0007958654637558523
----------------------------------------------------------------------------------------------
Net Worth: 52.88109694704191
Balance: 4.207620738599857
Bitcoins: 0.0008194789869653308
----------------------------------------------------------------------------------------------
Net Worth: 52.89930474578354
Balance: 3.155715553949893
Bitcoins: 0.000837182506830722
----------------------------------------------------------------------------------------------
Net Worth: 52.82295762545358
Balance: 2.3667866654624197
Bitcoins: 0.0008504805567531288
----------------------------------------------------------------------------------------------
Net Worth: 52.75628127868001
Balance: 1.7750899990968148
Bitcoins: 0.0008604672913711734
-------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------
Net Worth: 51.46564100537433
Balance: 0.12276064440835141
Bitcoins: 0.0008945888080176287
----------------------------------------------------------------------------------------------
Net Worth: 51.55657455991431
Balance: 0.09207048330626355
Bitcoins: 0.0008951226022776322
----------------------------------------------------------------------------------------------
Net Worth: 51.432593086353535
Balance: 0.06905286247969766
Bitcoins: 0.0008955239147615705
----------------------------------------------------------------------------------------------
Net Worth: 51.53724345133012
Balance: 0.05178964685977325
Bitcoins: 0.0008958242871324748
----------------------------------------------------------------------------------------------
Net Worth: 51.57837088432491
Balance: 0.038842235144829936
Bitcoins: 0.0008960493865974732
-------------------------------------------------------------------------

----------------------------------------------------------------------------------------------
Net Worth: 51.925100730189
Balance: 0.012643085991128896
Bitcoins: 0.0008962222260754143
----------------------------------------------------------------------------------------------
Net Worth: 51.88514882376723
Balance: 0.009482314493346671
Bitcoins: 0.000896276836000799
----------------------------------------------------------------------------------------------
Net Worth: 52.05070796056725
Balance: 0.007111735870010003
Bitcoins: 0.0008963176631466173
----------------------------------------------------------------------------------------------
Net Worth: 51.918080956348525
Balance: 0.005333801902507503
Bitcoins: 0.0008963483617377418
----------------------------------------------------------------------------------------------
Net Worth: 51.815666153298416
Balance: 0.004000351426880627
Bitcoins: 0.0008963714311931003
-----------------------------------------------------------------------